<a href="https://colab.research.google.com/github/susioesi/stock/blob/master/Finlab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install finlab > log.txt
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/libta-lib0_0.4.0-oneiric1_amd64.deb -qO libta.deb
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/ta-lib0-dev_0.4.0-oneiric1_amd64.deb -qO ta.deb
!dpkg -i libta.deb ta.deb
!pip install ta-lib

(Reading database ... 120908 files and directories currently installed.)
Preparing to unpack libta.deb ...
Unpacking libta-lib0 (0.4.0-oneiric1) over (0.4.0-oneiric1) ...
Preparing to unpack ta.deb ...
Unpacking ta-lib0-dev (0.4.0-oneiric1) over (0.4.0-oneiric1) ...
Setting up libta-lib0 (0.4.0-oneiric1) ...
Setting up ta-lib0-dev (0.4.0-oneiric1) ...
Processing triggers for man-db (2.10.2-1) ...
Processing triggers for libc-bin (2.35-0ubuntu3.4) ...
/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc_proxy.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_0.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_5.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbb.so.12 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc.so.2 is not a symbolic link



In [ ]:
from finlab import data, backtest, dataframe
import pandas as pd
import numpy as np
import plotly.express as px
import itertools

In [ ]:
category = ['光電業', '其他', '其他電子業', '化學工業', '半導體', '塑膠工業', '建材營造', '橡膠工業', '水泥工業', '汽車工業', '油電燃氣業', '生技醫療', '生技醫療業', '紡織纖維', '航運業', '貿易百貨', '資訊服務業', '農業科技', '通信網路業', '造紙工業', '金融', '電器電纜', '電子通路業', '電子零組件', '電機機械', '電腦及週邊', '食品工業']
#category = ['生技醫療', '生技醫療業', '電子零組件', '電腦及週邊']



with data.universe(market='TSE_OTC', category=category):
  rev = data.get('monthly_revenue:當月營收')
  monthly_revenue_grow = data.get("monthly_revenue:去年同月增減(%)")
  operating_profit = data.get("fundamental_features:營業利益率")#.deadline()
  net_income = data.get('fundamental_features:歸屬母公司淨利')#.deadline()
  net_income_r = data.get('fundamental_features:稅後淨利成長率')
  net_income_ratio = data.get('fundamental_features:稅後淨利率')

  eps = data.get('financial_statement:每股盈餘')#.deadline()
  df1 = data.get('financial_statement:投資活動之淨現金流入_流出')
  df2 = data.get('financial_statement:營業活動之淨現金流入_流出')
  fcf = (df1 + df2)#.rolling(4).mean()
  pe = data.get('price_earning_ratio:本益比')

  stock = data.get('financial_statement:存貨')
  stock_turnover = data.get('fundamental_features:存貨週轉率').replace([np.nan, np.inf], 0)
#season_revenue = ((rev * 1000).rolling(4).sum()).index_str_to_date().reindex(stock.index, method='ffill')
  season_revenue = data.get('financial_statement:營業收入淨額')
  #fcf = data.get('fundamental_features:自由現金流量')#.deadline()

def monthly_revenue_grow_modify():
  jan = [i for i in rev['2006':].index if str(i)[5:7] == '01']
  feb = [j for j in rev['2006':].index if str(j)[5:7] == '02']
  rev.loc[jan] += rev.loc[feb].set_index(rev.loc[jan].index)
  rev.loc[feb] = rev.loc[jan].set_index(rev.loc[feb].index)

  return (rev.diff(12) / rev.shift(12).abs() * 100)

'''
# 月營收考慮到年關因素
monthly_revenue_grow = monthly_revenue_grow_modify()
'''


'\n# 月營收考慮到年關因素\nmonthly_revenue_grow = monthly_revenue_grow_modify()\n'

In [ ]:
monthly_revenue_grow.tail(25)

symbol,1101,1102,1103,1104,1107,1108,1109,1110,1201,1203,...,9938,9939,9940,9941,9942,9944,9945,9946,9950,9951
date,,,,,,,,,,,,,,,,,,,,,
2021-12-10,-10.440833,13.403850,-5.339613,21.241605,NaN,-33.345518,8.708083,7.317301,3.004142,-19.266127,...,23.581492,13.807088,22.765785,10.423811,28.219983,3.728780,-18.115345,30.949029,39.297649,-16.179138
2022-01-10,-6.094106,6.773034,-16.874122,8.937706,NaN,-20.043601,-15.527323,17.835534,11.912984,0.695047,...,3.073079,13.814745,27.407377,11.625230,5.779704,5.494317,18.499902,81.725401,69.093598,-19.360010
2022-02-14,-6.094106,6.773034,-16.874122,8.937706,NaN,-20.043601,-15.527323,17.835534,11.912984,0.695047,...,3.073079,13.814745,27.407377,11.625230,5.779704,5.494317,18.499902,81.725401,69.093598,-19.360010
2022-03-10,20.261177,18.853146,-7.601436,-1.603048,NaN,-24.522792,-4.568165,-5.544672,-7.815703,2.497842,...,25.156674,17.938221,8.193260,8.950866,36.065258,15.834124,22.237117,82.874554,380.578739,-5.846733
2022-04-11,1.962572,16.467431,8.094472,2.882340,NaN,-1.120085,-44.421071,22.275361,5.496522,3.402506,...,-23.326343,6.707269,2.269671,10.055839,8.952225,7.959718,43.281053,-16.129171,167.728720,-16.163770
2022-05-10,-13.986436,11.229844,-9.252431,21.950115,NaN,-2.906851,-32.140423,-20.691165,-4.593507,9.571270,...,-37.201417,2.038156,-17.287708,8.420834,-21.784630,-13.802779,23.515176,-79.713494,113.533752,-29.125339
2022-06-10,-21.346765,4.051876,-16.742107,0.612198,NaN,-6.940698,4.247682,-36.738870,0.164040,9.320887,...,10.317962,14.671473,-10.602320,13.239934,9.938838,-1.029781,-27.183967,29.478363,209.102841,-16.078620
2022-07-11,7.843203,12.775121,8.545474,40.748164,NaN,2.595652,20.298129,-56.080414,2.179673,7.066730,...,-2.292413,15.409117,-29.355968,16.767388,5.694911,0.625194,75.518916,454.153023,283.817427,-13.566123
2022-08-10,21.424317,21.412029,-7.336398,17.556275,NaN,5.092646,13.262445,-15.558849,5.856234,4.120367,...,2.078779,13.389111,-25.769358,15.104402,6.954422,11.695347,-2.696772,9.849870,189.241117,-9.089661


## 營收成長性

In [ ]:
#with data.universe(market='TSE_OTC'):
#  rev = data.get('monthly_revenue:當月營收')
#  monthly_revenue_grow = data.get("monthly_revenue:去年同月增減(%)")


#monthly_revenue_grow = data.get("monthly_revenue:去年同月增減(%)").fillna(0)
#monthly_revenue_grow = monthly_revenue_grow_nodify().fillna(0)

monthly_revenue_grow_aa = ((monthly_revenue_grow > 0).sustain(6)) & (monthly_revenue_grow.average(6) > 25) & (monthly_revenue_grow.rise())
monthly_revenue_grow_a1 = ((monthly_revenue_grow > 0).sustain(6)) & ((monthly_revenue_grow.average(6) <= 25) & (monthly_revenue_grow.average(6) > 10)) & (monthly_revenue_grow.rise())
monthly_revenue_grow_a2 = ((monthly_revenue_grow > 0).sustain(6)) & (monthly_revenue_grow.average(6) > 25) & (monthly_revenue_grow < monthly_revenue_grow.shift()) & (monthly_revenue_grow.shift() > monthly_revenue_grow.shift(2)) & (monthly_revenue_grow.diff() / monthly_revenue_grow.shift().abs() * 100 > -50)#(monthly_revenue_grow.pct_change(fill_method=None) > -0.5)
monthly_revenue_grow_a = monthly_revenue_grow_a1 | monthly_revenue_grow_a2

monthly_revenue_grow_bb = (monthly_revenue_grow.rolling(6).min() <= 0) & ~(monthly_revenue_grow.fall().sustain(2)) & (monthly_revenue_grow > 0) & (monthly_revenue_grow.average(6) > 0)#~(monthly_revenue_grow_b | monthly_revenue_grow_c)
monthly_revenue_grow_b = ((monthly_revenue_grow.average(6) > 0)) & (monthly_revenue_grow.fall().sustain(2)) & (monthly_revenue_grow > 0)
monthly_revenue_grow_c = (monthly_revenue_grow.average(6) < 0) | (monthly_revenue_grow < 0)



empty_df = dataframe.FinlabDataFrame(pd.DataFrame(1, columns=monthly_revenue_grow.columns, index=monthly_revenue_grow.index))
empty_df &= ~(monthly_revenue_grow_aa | monthly_revenue_grow_a | monthly_revenue_grow_bb | monthly_revenue_grow_b | monthly_revenue_grow_c)
monthly_revenue_grow_bb |= empty_df

### pct_change() 的缺陷

In [ ]:
monthly_revenue_grow['1102'].tail(9)

date
2023-04-12    -0.09
2023-05-10   -17.69
2023-06-12    -9.52
2023-07-10   -14.21
2023-08-10   -13.66
2023-09-11    -8.47
2023-10-11    -4.32
2023-11-10   -11.77
2023-12-11      NaN
Name: 1102, dtype: float64

In [ ]:
monthly_revenue_grow.pct_change()['1102'].tail(9)

date
2023-04-12     -1.001724
2023-05-10    195.555556
2023-06-12     -0.461843
2023-07-10      0.492647
2023-08-10     -0.038705
2023-09-11     -0.379941
2023-10-11     -0.489965
2023-11-10      1.724537
2023-12-11      0.000000
Name: 1102, dtype: float64

In [ ]:
(monthly_revenue_grow.diff() / monthly_revenue_grow.shift().abs())['1102'].tail(9)

date
2023-04-12     -1.001724
2023-05-10   -195.555556
2023-06-12      0.461843
2023-07-10     -0.492647
2023-08-10      0.038705
2023-09-11      0.379941
2023-10-11      0.489965
2023-11-10     -1.724537
2023-12-11           NaN
Name: 1102, dtype: float64

### 重疊驗證

In [ ]:
condition_list = {'營收成長 AA': monthly_revenue_grow_aa, '營收成長 A': monthly_revenue_grow_a, '營收成長 BB': monthly_revenue_grow_bb, '營收成長 B': monthly_revenue_grow_b, '營收成長 C': monthly_revenue_grow_c}

for c in condition_list:
  print(c + ' 股票數: ' + str(len(condition_list[c].iloc[-1][condition_list[c].iloc[-1]])))

for c in list(itertools.combinations(condition_list, 2)):
  for i in np.arange(1, 5, 1):
    con = condition_list[c[0]] & condition_list[c[1]]
    print(c[0] + ' 與 ' + c[1] + ' 過去 ' + str(i) + ' 期重複股票數: ' + str(len(con.iloc[-i][con.iloc[-i]])))

營收成長 AA 股票數: 0
營收成長 A 股票數: 0
營收成長 BB 股票數: 792
營收成長 B 股票數: 1
營收成長 C 股票數: 886
營收成長 AA 與 營收成長 A 過去 1 期重複股票數: 0
營收成長 AA 與 營收成長 A 過去 2 期重複股票數: 0
營收成長 AA 與 營收成長 A 過去 3 期重複股票數: 0
營收成長 AA 與 營收成長 A 過去 4 期重複股票數: 0
營收成長 AA 與 營收成長 BB 過去 1 期重複股票數: 0
營收成長 AA 與 營收成長 BB 過去 2 期重複股票數: 0
營收成長 AA 與 營收成長 BB 過去 3 期重複股票數: 0
營收成長 AA 與 營收成長 BB 過去 4 期重複股票數: 0
營收成長 AA 與 營收成長 B 過去 1 期重複股票數: 0
營收成長 AA 與 營收成長 B 過去 2 期重複股票數: 0
營收成長 AA 與 營收成長 B 過去 3 期重複股票數: 0
營收成長 AA 與 營收成長 B 過去 4 期重複股票數: 0
營收成長 AA 與 營收成長 C 過去 1 期重複股票數: 0
營收成長 AA 與 營收成長 C 過去 2 期重複股票數: 0
營收成長 AA 與 營收成長 C 過去 3 期重複股票數: 0
營收成長 AA 與 營收成長 C 過去 4 期重複股票數: 0
營收成長 A 與 營收成長 BB 過去 1 期重複股票數: 0
營收成長 A 與 營收成長 BB 過去 2 期重複股票數: 0
營收成長 A 與 營收成長 BB 過去 3 期重複股票數: 0
營收成長 A 與 營收成長 BB 過去 4 期重複股票數: 0
營收成長 A 與 營收成長 B 過去 1 期重複股票數: 0
營收成長 A 與 營收成長 B 過去 2 期重複股票數: 0
營收成長 A 與 營收成長 B 過去 3 期重複股票數: 0
營收成長 A 與 營收成長 B 過去 4 期重複股票數: 0
營收成長 A 與 營收成長 C 過去 1 期重複股票數: 0
營收成長 A 與 營收成長 C 過去 2 期重複股票數: 0
營收成長 A 與 營收成長 C 過去 3 期重複股票數: 0
營收成長 A 與 營收成長 C 過去 4 期重複股票數: 0
營收成長 BB 與 營收成長 B 過去 1 期重複股票數: 0


### 報酬率

In [ ]:
import numpy as np

return_dataset = {}
s_list = {
    'AA':monthly_revenue_grow_aa,
    'A':monthly_revenue_grow_a,
    'BB':monthly_revenue_grow_bb,
    'B':monthly_revenue_grow_b,
    'C':monthly_revenue_grow_c,
}

for k in s_list:
  r = backtest.sim(position=s_list[k], upload=False)
  return_series = r.creturn
  return_dataset[f"{k}"] = return_series
df = pd.DataFrame(return_dataset)

fig = px.line(df, height=800, title='Monthly revenue use origin YOY')
fig.show()

## 營業利益率

In [ ]:
#with data.universe(market='TSE_OTC'):
#  operating_profit = data.get("fundamental_features:營業利益率")

#operating_profit_stable = ((((operating_profit - operating_profit.shift(1)) / abs(operating_profit.shift(1))).rolling(3).min()) * 100).fillna(0) > -20
#operating_profit = operating_profit.index_str_to_date().reindex(rev.index_str_to_date().index, method='ffill')
operating_profit_stable = (((operating_profit.diff() / operating_profit.shift().abs()).rolling(3).min()) * 100) > -20
#operating_profit = operating_profit.deadline()
#operating_profit_stable = operating_profit_stable.deadline()

operating_profit_aa_1 = operating_profit_stable & (operating_profit.average(4) > 15)
operating_profit_aa_2 = operating_profit_stable & ((operating_profit.average(4) <= 15) & ((operating_profit.average(4) > 10)) & operating_profit.rise())
operating_profit_aa = operating_profit_aa_1 | operating_profit_aa_2


operating_profit_a_1 = operating_profit_stable & ((operating_profit.average(4) <= 15) & (operating_profit.average(4) > 10)) & operating_profit.fall() # 考慮到 AA_2 條件，A_1 會涵蓋其包含的股票，故排除最近一季呈現上升趨勢
operating_profit_a_2 = operating_profit_stable & ((operating_profit.average(4) <= 10) & (operating_profit.average(4) > 5)) & (operating_profit.rise())
operating_profit_a =  operating_profit_a_1 | operating_profit_a_2


operating_profit_bb = (~operating_profit_stable) & ((operating_profit.diff()/ operating_profit.shift().abs()) * 100 > -20) & (operating_profit.average(4) > 5) & (operating_profit > 0) # 考慮到 B 條件，不論漲跌幅，過去四季平均營益率皆在 5% 以下。BB 條件須排除
                                                                                                            # 考慮到 C 條件，最近一季為負。BB 條件須排除

operating_profit_b_1 = ((operating_profit.diff()/ operating_profit.shift().abs()) * 100 <= -20)# & (operating_profit.average(4) > 0) # 考慮到 B_2 條件，不論帳跌幅，過去四季平均在 5% 以下。B_1 條件排除過去四季平均在 5% 以下條件。
operating_profit_b_2 = (operating_profit.average(4) <= 5) #& (operating_profit.average(4) > 0) # 考慮到 C 條件，過去四季平均為負。 避免股票同時屬於 B 條件與 C 條件，新增過去四季平均大於 0 的條件
operating_profit_b = (operating_profit_b_1 | operating_profit_b_2) & (operating_profit > 0 ) # 考慮到 C 條件，最近一季為負。這裡避免 B 條件被歸納到 C，新增最近一季為正的條件
operating_profit_b &= (operating_profit.average(4) > 0)

operating_profit_c = (operating_profit.average(4) <= 0) | (operating_profit <= 0)


empty_df = dataframe.FinlabDataFrame(pd.DataFrame(1, columns=operating_profit_stable.columns, index=operating_profit_stable.index))
empty_df &= ~(operating_profit_aa | operating_profit_a | operating_profit_bb | operating_profit_b | operating_profit_c)
operating_profit_bb |= empty_df


#operating_profit_aa = operating_profit_aa.deadline()
#operating_profit_a = operating_profit_a.deadline()
#operating_profit_bb = operating_profit_bb.deadline()
#operating_profit_b = operating_profit_b.deadline()
#operating_profit_c = operating_profit_c.deadline()

### 重疊驗證

In [ ]:
condition_list = {'營業利益率 AA': operating_profit_aa, '營業利益率 A': operating_profit_a, '營業利益率 BB': operating_profit_bb, '營業利益率 B': operating_profit_b, '營業利益率 C': operating_profit_c}

for c in condition_list:
  print(c + ' 股票數: ' + str(len(condition_list[c].iloc[-1][condition_list[c].iloc[-1]])))

for c in list(itertools.combinations(condition_list, 2)):
  for i in np.arange(1, 5, 1):
    con = condition_list[c[0]] & condition_list[c[1]]
    print(c[0] + ' 與 ' + c[1] + ' 過去 ' + str(i) + ' 期重複股票數: ' + str(len(con.iloc[-i][con.iloc[-i]])))

營業利益率 AA 股票數: 214
營業利益率 A 股票數: 79
營業利益率 BB 股票數: 388
營業利益率 B 股票數: 382
營業利益率 C 股票數: 588
營業利益率 AA 與 營業利益率 A 過去 1 期重複股票數: 0
營業利益率 AA 與 營業利益率 A 過去 2 期重複股票數: 0
營業利益率 AA 與 營業利益率 A 過去 3 期重複股票數: 0
營業利益率 AA 與 營業利益率 A 過去 4 期重複股票數: 0
營業利益率 AA 與 營業利益率 BB 過去 1 期重複股票數: 0
營業利益率 AA 與 營業利益率 BB 過去 2 期重複股票數: 0
營業利益率 AA 與 營業利益率 BB 過去 3 期重複股票數: 0
營業利益率 AA 與 營業利益率 BB 過去 4 期重複股票數: 0
營業利益率 AA 與 營業利益率 B 過去 1 期重複股票數: 0
營業利益率 AA 與 營業利益率 B 過去 2 期重複股票數: 0
營業利益率 AA 與 營業利益率 B 過去 3 期重複股票數: 0
營業利益率 AA 與 營業利益率 B 過去 4 期重複股票數: 0
營業利益率 AA 與 營業利益率 C 過去 1 期重複股票數: 0
營業利益率 AA 與 營業利益率 C 過去 2 期重複股票數: 0
營業利益率 AA 與 營業利益率 C 過去 3 期重複股票數: 0
營業利益率 AA 與 營業利益率 C 過去 4 期重複股票數: 0
營業利益率 A 與 營業利益率 BB 過去 1 期重複股票數: 0
營業利益率 A 與 營業利益率 BB 過去 2 期重複股票數: 0
營業利益率 A 與 營業利益率 BB 過去 3 期重複股票數: 0
營業利益率 A 與 營業利益率 BB 過去 4 期重複股票數: 0
營業利益率 A 與 營業利益率 B 過去 1 期重複股票數: 0
營業利益率 A 與 營業利益率 B 過去 2 期重複股票數: 0
營業利益率 A 與 營業利益率 B 過去 3 期重複股票數: 0
營業利益率 A 與 營業利益率 B 過去 4 期重複股票數: 0
營業利益率 A 與 營業利益率 C 過去 1 期重複股票數: 0
營業利益率 A 與 營業利益率 C 過去 2 期重複股票數: 0
營業利益率 A 與 營業利益率 C 過去 3 期重複股票數: 0

### 報酬率

In [ ]:
ret = []
return_dataset = {}
s_list = {'AA':operating_profit_aa, 'A':operating_profit_a, 'BB':operating_profit_bb, 'B':operating_profit_b, 'C':operating_profit_c}

for k in s_list:
  #s_list[k].index_str_to_date().reindex(rev.index_str_to_date().index, method='ffill')
  r = backtest.sim(position=s_list[k], upload=False)

  return_series = r.creturn
  return_dataset[f"{k}"] = return_series
df = pd.DataFrame(return_dataset)


fig = px.line(df, height=800, title='operation profit grow')

fig.show()

## 獲利成長性

In [ ]:
net_income_grow = net_income_r#(net_income.diff(4) / net_income.shift(4).abs() * 100)
net_income_grow_stable = net_income_grow.pct_change(fill_method=None) * 100 > -50

net_income_grow_aa_1 = ((net_income_grow > 0).sustain(3)) & (net_income_grow.rise())
net_income_grow_aa_2 = (net_income_grow > 50).sustain(3)
net_income_grow_aa = (net_income_grow_aa_1 | net_income_grow_aa_2)

net_income_grow_a = (((net_income_grow > 0).sustain(2)) & net_income_grow_stable) & ~(net_income_grow_aa)#~((net_income_grow > 0).sustain(3))
net_income_grow_a &= ~((net_income_grow < 0).sustain(4, 2)) # 考慮到 B_2 條件，A 須排除最近四季出現兩季負數的情況
net_income_grow_a &= ~(net_income_grow.fall().sustain(2))

net_income_grow_bb_1 = ((net_income_grow > 0).sustain(2)) & ~(net_income_grow_stable)
net_income_grow_bb_1 &= ~((net_income_grow > 50).sustain(3))  # BB_1 的近兩季為正但最近一季出現 50% 跌幅會涵蓋符合 AA_2 的股票，故須排除近三季節大於 50% 的情況
net_income_grow_bb_1 &= ~(net_income_grow.fall().sustain(2)) # 考慮到 B3 條件，BB_1 須排除連續三季下降的情況
net_income_grow_bb_2 = (net_income_grow.shift() < 0) & (net_income_grow > 0)
net_income_grow_bb = (net_income_grow_bb_1 | net_income_grow_bb_2)
net_income_grow_bb &= (net_income_grow > 0) # 考慮到 B_1 條件，BB 須排除最近一季出現負數的情況
net_income_grow_bb &= ~((net_income_grow < 0).sustain(4, 2))  # 考慮到 B_2 條件，BB 須排除最近四季出現兩季負數的情況

net_income_grow_b_1 = net_income_grow < 0
net_income_grow_b_2 = (net_income_grow < 0).sustain(4, 2)
net_income_grow_b_3 = (net_income_grow.fall().sustain(2)) & (net_income_grow < 50)
net_income_grow_b = (net_income_grow_b_1 | net_income_grow_b_2 | net_income_grow_b_3) & ~((net_income_grow < 0).sustain(2)) # 靠慮到 C 條件， B 條件須排除近兩季為負數的情況

net_income_grow_c = (net_income_grow < 0).sustain(2)

#net_income_grow_aa = net_income_grow_aa.index_str_to_date().reindex(rev.index_str_to_date().index, method='ffill')
#net_income_grow_a = net_income_grow_a.index_str_to_date().reindex(rev.index_str_to_date().index, method='ffill')
#net_income_grow_bb = net_income_grow_bb.index_str_to_date().reindex(rev.index_str_to_date().index, method='ffill')
#net_income_grow_b = net_income_grow_b.index_str_to_date().reindex(rev.index_str_to_date().index, method='ffill')
#net_income_grow_c = net_income_grow_c.index_str_to_date().reindex(rev.index_str_to_date().index, method='ffill')

### 重疊驗證

In [ ]:
condition_list = {'獲利成長性 AA': net_income_grow_aa, '獲利成長性 A': net_income_grow_a, '獲利成長性 BB': net_income_grow_bb, '獲利成長性 B': net_income_grow_b, '獲利成長性 C': net_income_grow_c}

for c in condition_list:
  print(c + ' 股票數: ' + str(len(condition_list[c].iloc[-1][condition_list[c].iloc[-1]])))

for c in list(itertools.combinations(condition_list, 2)):
  for i in np.arange(1, 5, 1):
    con = condition_list[c[0]] & condition_list[c[1]]
    print(c[0] + ' 與 ' + c[1] + ' 過去 ' + str(i) + ' 期重複股票數: ' + str(len(con.iloc[-i][con.iloc[-i]])))


獲利成長性 AA 股票數: 144
獲利成長性 A 股票數: 48
獲利成長性 BB 股票數: 81
獲利成長性 B 股票數: 666
獲利成長性 C 股票數: 646
獲利成長性 AA 與 獲利成長性 A 過去 1 期重複股票數: 0
獲利成長性 AA 與 獲利成長性 A 過去 2 期重複股票數: 0
獲利成長性 AA 與 獲利成長性 A 過去 3 期重複股票數: 0
獲利成長性 AA 與 獲利成長性 A 過去 4 期重複股票數: 0
獲利成長性 AA 與 獲利成長性 BB 過去 1 期重複股票數: 0
獲利成長性 AA 與 獲利成長性 BB 過去 2 期重複股票數: 0
獲利成長性 AA 與 獲利成長性 BB 過去 3 期重複股票數: 0
獲利成長性 AA 與 獲利成長性 BB 過去 4 期重複股票數: 0
獲利成長性 AA 與 獲利成長性 B 過去 1 期重複股票數: 0
獲利成長性 AA 與 獲利成長性 B 過去 2 期重複股票數: 0
獲利成長性 AA 與 獲利成長性 B 過去 3 期重複股票數: 0
獲利成長性 AA 與 獲利成長性 B 過去 4 期重複股票數: 0
獲利成長性 AA 與 獲利成長性 C 過去 1 期重複股票數: 0
獲利成長性 AA 與 獲利成長性 C 過去 2 期重複股票數: 0
獲利成長性 AA 與 獲利成長性 C 過去 3 期重複股票數: 0
獲利成長性 AA 與 獲利成長性 C 過去 4 期重複股票數: 0
獲利成長性 A 與 獲利成長性 BB 過去 1 期重複股票數: 0
獲利成長性 A 與 獲利成長性 BB 過去 2 期重複股票數: 0
獲利成長性 A 與 獲利成長性 BB 過去 3 期重複股票數: 0
獲利成長性 A 與 獲利成長性 BB 過去 4 期重複股票數: 0
獲利成長性 A 與 獲利成長性 B 過去 1 期重複股票數: 0
獲利成長性 A 與 獲利成長性 B 過去 2 期重複股票數: 0
獲利成長性 A 與 獲利成長性 B 過去 3 期重複股票數: 0
獲利成長性 A 與 獲利成長性 B 過去 4 期重複股票數: 0
獲利成長性 A 與 獲利成長性 C 過去 1 期重複股票數: 0
獲利成長性 A 與 獲利成長性 C 過去 2 期重複股票數: 0
獲利成長性 A 與 獲利成長性 C 過去 3 期重複股票數: 0


### 報酬率

In [ ]:
ret = []
return_dataset = {}
s_list = {'AA':net_income_grow_aa, 'A':net_income_grow_a, 'BB':net_income_grow_bb, 'B':net_income_grow_b, 'C':net_income_grow_c}

for k in s_list:
  #s_list[k].index_str_to_date().reindex(rev.index_str_to_date().index, method='ffill')
  r = backtest.sim(position=s_list[k], upload=False)

  return_series = r.creturn
  return_dataset[f"{k}"] = return_series
df = pd.DataFrame(return_dataset)

fig = px.line(df, height=800, title='Net income grow')

fig.show()

## 獲利能力

In [ ]:
eps_aa = (eps.rolling(4).sum() > 5) & (eps > 0)
eps_a = (eps.rolling(4).sum() <= 5) & (eps.rolling(4).sum() > 3) & (eps > 0)
eps_bb = (eps.rolling(4).sum() <= 3) & (eps.rolling(4).sum() > 1) & (eps > 0)
eps_b_1 = (eps.rolling(4).sum() <= 1)
eps_b_2 = (eps < 0)
eps_b = (eps_b_1 | eps_b_2) & (eps.rolling(4).sum() > 0)
eps_c = eps.rolling(4).sum() <= 0

#eps_aa = eps_aa.index_str_to_date().reindex(rev.index_str_to_date().index, method='ffill')
#eps_a = eps_a.index_str_to_date().reindex(rev.index_str_to_date().index, method='ffill')
#eps_bb = eps_bb.index_str_to_date().reindex(rev.index_str_to_date().index, method='ffill')
#eps_b = eps_b.index_str_to_date().reindex(rev.index_str_to_date().index, method='ffill')
#eps_c = eps_c.index_str_to_date().reindex(rev.index_str_to_date().index, method='ffill')

### 重疊驗證

In [ ]:
condition_list = {'獲利能力 AA': eps_aa, '獲利能力 A': eps_a, '獲利能力 BB': eps_bb, '獲利能力 B': eps_b, '獲利能力 C': eps_c}

for c in condition_list:
  print(c + ' 股票數: ' + str(len(condition_list[c].iloc[-1][condition_list[c].iloc[-1]])))

for c in list(itertools.combinations(condition_list, 2)):
  for i in np.arange(1, 5, 1):
    con = condition_list[c[0]] & condition_list[c[1]]
    print(c[0] + ' 與 ' + c[1] + ' 過去 ' + str(i) + ' 期重複股票數: ' + str(len(con.iloc[-i][con.iloc[-i]])))

獲利能力 AA 股票數: 306
獲利能力 A 股票數: 201
獲利能力 BB 股票數: 382
獲利能力 B 股票數: 283
獲利能力 C 股票數: 411
獲利能力 AA 與 獲利能力 A 過去 1 期重複股票數: 0
獲利能力 AA 與 獲利能力 A 過去 2 期重複股票數: 0
獲利能力 AA 與 獲利能力 A 過去 3 期重複股票數: 0
獲利能力 AA 與 獲利能力 A 過去 4 期重複股票數: 0
獲利能力 AA 與 獲利能力 BB 過去 1 期重複股票數: 0
獲利能力 AA 與 獲利能力 BB 過去 2 期重複股票數: 0
獲利能力 AA 與 獲利能力 BB 過去 3 期重複股票數: 0
獲利能力 AA 與 獲利能力 BB 過去 4 期重複股票數: 0
獲利能力 AA 與 獲利能力 B 過去 1 期重複股票數: 0
獲利能力 AA 與 獲利能力 B 過去 2 期重複股票數: 0
獲利能力 AA 與 獲利能力 B 過去 3 期重複股票數: 0
獲利能力 AA 與 獲利能力 B 過去 4 期重複股票數: 0
獲利能力 AA 與 獲利能力 C 過去 1 期重複股票數: 0
獲利能力 AA 與 獲利能力 C 過去 2 期重複股票數: 0
獲利能力 AA 與 獲利能力 C 過去 3 期重複股票數: 0
獲利能力 AA 與 獲利能力 C 過去 4 期重複股票數: 0
獲利能力 A 與 獲利能力 BB 過去 1 期重複股票數: 0
獲利能力 A 與 獲利能力 BB 過去 2 期重複股票數: 0
獲利能力 A 與 獲利能力 BB 過去 3 期重複股票數: 0
獲利能力 A 與 獲利能力 BB 過去 4 期重複股票數: 0
獲利能力 A 與 獲利能力 B 過去 1 期重複股票數: 0
獲利能力 A 與 獲利能力 B 過去 2 期重複股票數: 0
獲利能力 A 與 獲利能力 B 過去 3 期重複股票數: 0
獲利能力 A 與 獲利能力 B 過去 4 期重複股票數: 0
獲利能力 A 與 獲利能力 C 過去 1 期重複股票數: 0
獲利能力 A 與 獲利能力 C 過去 2 期重複股票數: 0
獲利能力 A 與 獲利能力 C 過去 3 期重複股票數: 0
獲利能力 A 與 獲利能力 C 過去 4 期重複股票數: 0
獲利能力 BB 與 獲利能力 B 過去 1 期重複股

### 報酬率

In [ ]:
ret = []
return_dataset = {}
s_list = {'AA':eps_aa, 'A':eps_a, 'BB':eps_bb, 'B':eps_b, 'C':eps_c}

for k in s_list:
  #s_list[k].index_str_to_date().reindex(rev.index_str_to_date().index, method='ffill')
  r = backtest.sim(position=s_list[k], upload=False)

  return_series = r.creturn
  return_dataset[f"{k}"] = return_series
df = pd.DataFrame(return_dataset)

fig = px.line(df, height=800, title='EPS')

fig.show()

## 存貨周轉率

In [ ]:


stock_low = (stock_turnover <= 0) | (stock_turnover > 10) | ((stock / season_revenue) <= 0.04)
#stock_turnover &= ~stock_low
stock_turnover_stable = (stock_turnover.diff() / stock_turnover.shift().abs()).rolling(3).min() * 100 > -20
stock_turnover_cumulate_loss_gt_m20 = (stock_turnover.fall().sustain(3, 2)) & (stock_turnover.pct_change()[stock_turnover.pct_change() < 0].rolling(2).sum() * 100 < -20)

stock_turnover_aa = (~stock_low) & stock_turnover_stable & (stock_turnover.average(4) > 1.5) & ~(stock_turnover_cumulate_loss_gt_m20)
stock_turnover_a = (~stock_low) & stock_turnover_stable & (stock_turnover.average(4) <= 1.5) & ~(stock_turnover_cumulate_loss_gt_m20)
stock_turnover_bb = (~stock_low) & stock_turnover_cumulate_loss_gt_m20 & (stock_turnover_stable)
stock_turnover_b = (~stock_low) & ~(stock_turnover_stable) & (stock_turnover.pct_change() * 100 > -20)
stock_turnover_c = (~stock_low) & (stock_turnover.pct_change() * 100 <= -20)

#stock_turnover_aa = stock_turnover_aa.index_str_to_date().reindex(rev.index_str_to_date().index, method='ffill')
#stock_turnover_a = stock_turnover_a.index_str_to_date().reindex(rev.index_str_to_date().index, method='ffill')
#stock_turnover_bb = stock_turnover_bb.index_str_to_date().reindex(rev.index_str_to_date().index, method='ffill')
#stock_turnover_b = stock_turnover_b.index_str_to_date().reindex(rev.index_str_to_date().index, method='ffill')
#stock_turnover_c = stock_turnover_c.index_str_to_date().reindex(rev.index_str_to_date().index, method='ffill')

### 重疊驗證

In [ ]:
condition_list = {'存貨周轉率 AA': stock_turnover_aa, '存貨周轉率 A': stock_turnover_a, '存貨周轉率 BB': stock_turnover_bb, '存貨周轉率 B': stock_turnover_b, '存貨周轉率 C': stock_turnover_c}

for c in condition_list:
  print(c + ' 股票數: ' + str(len(condition_list[c].iloc[-1][condition_list[c].iloc[-1]])))

for c in list(itertools.combinations(condition_list, 2)):
  for i in np.arange(1, 5, 1):
    con = condition_list[c[0]] & condition_list[c[1]]
    print(c[0] + ' 與 ' + c[1] + ' 過去 ' + str(i) + ' 期重複股票數: ' + str(len(con.iloc[-i][con.iloc[-i]])))

存貨周轉率 AA 股票數: 216
存貨周轉率 A 股票數: 750
存貨周轉率 BB 股票數: 17
存貨周轉率 B 股票數: 328
存貨周轉率 C 股票數: 134
存貨周轉率 AA 與 存貨周轉率 A 過去 1 期重複股票數: 0
存貨周轉率 AA 與 存貨周轉率 A 過去 2 期重複股票數: 0
存貨周轉率 AA 與 存貨周轉率 A 過去 3 期重複股票數: 0
存貨周轉率 AA 與 存貨周轉率 A 過去 4 期重複股票數: 0
存貨周轉率 AA 與 存貨周轉率 BB 過去 1 期重複股票數: 0
存貨周轉率 AA 與 存貨周轉率 BB 過去 2 期重複股票數: 0
存貨周轉率 AA 與 存貨周轉率 BB 過去 3 期重複股票數: 0
存貨周轉率 AA 與 存貨周轉率 BB 過去 4 期重複股票數: 0
存貨周轉率 AA 與 存貨周轉率 B 過去 1 期重複股票數: 0
存貨周轉率 AA 與 存貨周轉率 B 過去 2 期重複股票數: 0
存貨周轉率 AA 與 存貨周轉率 B 過去 3 期重複股票數: 0
存貨周轉率 AA 與 存貨周轉率 B 過去 4 期重複股票數: 0
存貨周轉率 AA 與 存貨周轉率 C 過去 1 期重複股票數: 0
存貨周轉率 AA 與 存貨周轉率 C 過去 2 期重複股票數: 0
存貨周轉率 AA 與 存貨周轉率 C 過去 3 期重複股票數: 0
存貨周轉率 AA 與 存貨周轉率 C 過去 4 期重複股票數: 0
存貨周轉率 A 與 存貨周轉率 BB 過去 1 期重複股票數: 0
存貨周轉率 A 與 存貨周轉率 BB 過去 2 期重複股票數: 0
存貨周轉率 A 與 存貨周轉率 BB 過去 3 期重複股票數: 0
存貨周轉率 A 與 存貨周轉率 BB 過去 4 期重複股票數: 0
存貨周轉率 A 與 存貨周轉率 B 過去 1 期重複股票數: 0
存貨周轉率 A 與 存貨周轉率 B 過去 2 期重複股票數: 0
存貨周轉率 A 與 存貨周轉率 B 過去 3 期重複股票數: 0
存貨周轉率 A 與 存貨周轉率 B 過去 4 期重複股票數: 0
存貨周轉率 A 與 存貨周轉率 C 過去 1 期重複股票數: 0
存貨周轉率 A 與 存貨周轉率 C 過去 2 期重複股票數: 0
存貨周轉率 A 與 存貨周轉率 C 過去 3 期重複股票數: 0

### 報酬率

In [ ]:
ret = []
return_dataset = {}
s_list = {'AA':stock_turnover_aa, 'A':stock_turnover_a, 'BB':stock_turnover_bb, 'B':stock_turnover_b, 'C':stock_turnover_c}

for k in s_list:
  #s_list[k].index_str_to_date().reindex(rev.index_str_to_date().index, method='ffill')
  r = backtest.sim(position=s_list[k], upload=False)

  return_series = r.creturn
  return_dataset[f"{k}"] = return_series
df = pd.DataFrame(return_dataset)

fig = px.line(df, height=800, title='Stock turnover')

fig.show()

Daily usage: 707.8 / 5000 MB - etl:is_flagged_stock


## 自由現金流

In [ ]:
fcf_aa = fcf.rolling(6).min() > 0
fcf_a = (fcf.rolling(6).sum() > 0) & (fcf.rolling(4).sum() > 0) & ~(fcf_aa)
fcf_bb = (fcf.rolling(6).sum() <= 0) & (fcf.rolling(4).sum() > 0)# & ~(fcf.rolling(6).min() > 0)
fcf_b = (fcf.rolling(6).sum() > 0) & (fcf.rolling(4).sum() <= 0)# & ~(fcf.rolling(6).min() > 0)
fcf_c = (fcf.rolling(6).sum() <= 0) & (fcf.rolling(4).sum() <= 0)# & ~(fcf.rolling(6).min() > 0)



### 重疊驗證

In [ ]:
condition_list = {'自由現金流 AA': fcf_aa, '自由現金流 A': fcf_a, '自由現金流 BB': fcf_bb, '自由現金流 B': fcf_b, '自由現金流 C': fcf_c}

for c in condition_list:
  print(c + ' 股票數: ' + str(len(condition_list[c].iloc[-1][condition_list[c].iloc[-1]])))

for c in list(itertools.combinations(condition_list, 2)):
  for i in np.arange(1, 5, 1):
    con = condition_list[c[0]] & condition_list[c[1]]
    print(c[0] + ' 與 ' + c[1] + ' 過去 ' + str(i) + ' 期重複股票數: ' + str(len(con.iloc[-i][con.iloc[-i]])))

自由現金流 AA 股票數: 156
自由現金流 A 股票數: 778
自由現金流 BB 股票數: 92
自由現金流 B 股票數: 141
自由現金流 C 股票數: 408
自由現金流 AA 與 自由現金流 A 過去 1 期重複股票數: 0
自由現金流 AA 與 自由現金流 A 過去 2 期重複股票數: 0
自由現金流 AA 與 自由現金流 A 過去 3 期重複股票數: 0
自由現金流 AA 與 自由現金流 A 過去 4 期重複股票數: 0
自由現金流 AA 與 自由現金流 BB 過去 1 期重複股票數: 0
自由現金流 AA 與 自由現金流 BB 過去 2 期重複股票數: 0
自由現金流 AA 與 自由現金流 BB 過去 3 期重複股票數: 0
自由現金流 AA 與 自由現金流 BB 過去 4 期重複股票數: 0
自由現金流 AA 與 自由現金流 B 過去 1 期重複股票數: 0
自由現金流 AA 與 自由現金流 B 過去 2 期重複股票數: 0
自由現金流 AA 與 自由現金流 B 過去 3 期重複股票數: 0
自由現金流 AA 與 自由現金流 B 過去 4 期重複股票數: 0
自由現金流 AA 與 自由現金流 C 過去 1 期重複股票數: 0
自由現金流 AA 與 自由現金流 C 過去 2 期重複股票數: 0
自由現金流 AA 與 自由現金流 C 過去 3 期重複股票數: 0
自由現金流 AA 與 自由現金流 C 過去 4 期重複股票數: 0
自由現金流 A 與 自由現金流 BB 過去 1 期重複股票數: 0
自由現金流 A 與 自由現金流 BB 過去 2 期重複股票數: 0
自由現金流 A 與 自由現金流 BB 過去 3 期重複股票數: 0
自由現金流 A 與 自由現金流 BB 過去 4 期重複股票數: 0
自由現金流 A 與 自由現金流 B 過去 1 期重複股票數: 0
自由現金流 A 與 自由現金流 B 過去 2 期重複股票數: 0
自由現金流 A 與 自由現金流 B 過去 3 期重複股票數: 0
自由現金流 A 與 自由現金流 B 過去 4 期重複股票數: 0
自由現金流 A 與 自由現金流 C 過去 1 期重複股票數: 0
自由現金流 A 與 自由現金流 C 過去 2 期重複股票數: 0
自由現金流 A 與 自由現金流 C 過去 3 期重複股票數: 0

### 報酬率

In [ ]:
ret = []
return_dataset = {}
s_list = {'AA':fcf_aa, 'A':fcf_a, 'BB':fcf_bb, 'B':fcf_b, 'C':fcf_c}

for k in s_list:
  #s_list[k].index_str_to_date().reindex(rev.index_str_to_date().index, method='ffill')
  r = backtest.sim(position=s_list[k], upload=False)

  return_series = r.creturn
  return_dataset[f"{k}"] = return_series
df = pd.DataFrame(return_dataset)

fig = px.line(df, height=600, title='Free cash flow')

fig.show()

In [ ]:
substract_level_c = monthly_revenue_grow_c | operating_profit_c | net_income_grow_c | eps_c | fcf_c | stock_turnover_c
select_low_stock = ~substract_level_c * stock_low * ((4 * (monthly_revenue_grow_aa + operating_profit_aa + net_income_grow_aa + eps_aa + fcf_aa) +
                3 * (monthly_revenue_grow_a + operating_profit_a + net_income_grow_a + eps_a + fcf_a) +
                2 * (monthly_revenue_grow_bb + operating_profit_bb + net_income_grow_bb + eps_bb + fcf_bb) +
                1 * (monthly_revenue_grow_b + operating_profit_b + net_income_grow_b + eps_b + fcf_b)) / 5)
select_normal = ~substract_level_c * ~(stock_low) * ((4 * (monthly_revenue_grow_aa + operating_profit_aa + net_income_grow_aa + eps_aa + stock_turnover_aa + fcf_aa) +
              3 * (monthly_revenue_grow_a + operating_profit_a + net_income_grow_a + eps_a + stock_turnover_a + fcf_a) +
              2 * (monthly_revenue_grow_bb + operating_profit_bb + net_income_grow_bb + eps_bb + stock_turnover_bb + fcf_bb) +
              1 * (monthly_revenue_grow_b + operating_profit_b + net_income_grow_b + eps_b + stock_turnover_b + fcf_b)) / 6)

s = select_normal + select_low_stock

In [ ]:
p = ((s > 3) & (s - s.shift() >= -0.3))
r = backtest.sim(position=p)

## 雷達圖

In [ ]:
# 導入函式庫
from finlab.plot import plot_tw_stock_radar
# 製作客製化雷達圖
monthly_revenue_grow_t = monthly_revenue_grow_aa * 4 + monthly_revenue_grow_a * 3 +\
                       monthly_revenue_grow_bb * 2 + monthly_revenue_grow_b * 1 +\
                       monthly_revenue_grow_c * 0
net_income_grow_t = net_income_grow_aa * 4 + net_income_grow_a * 3 +\
                  net_income_grow_bb * 2 + net_income_grow_b * 1 +\
                  net_income_grow_c * 0
operating_profit_t = operating_profit_aa * 4 + operating_profit_a * 3 +\
                   operating_profit_bb * 2 + operating_profit_b * 1 +\
                   operating_profit_c * 0
eps_t = eps_aa * 4 + eps_a * 3 + eps_bb * 2 + eps_b * 1 + eps_c * 0
stock_turnover_t = stock_turnover_aa * 4 + stock_turnover_a * 3 + stock_turnover_bb * 2 +\
                 stock_turnover_b * 1 + stock_turnover_c * 0
fcf_t = fcf_aa * 4 + fcf_a * 3 + fcf_bb * 2 + fcf_b * 1 + fcf_c * 0
custom_data = pd.DataFrame({
    '營收成長性': monthly_revenue_grow_t.iloc[-1],
    '獲利成長性': net_income_grow_t.iloc[-1],
    '營業利益率': operating_profit_t.iloc[-1],
    '獲利能力': eps_t.iloc[-1],
    '存貨週轉率': stock_turnover_t.iloc[-1],
    '自由現金流': fcf_t.iloc[-1]}
)
custom_data.index.name = 'stock_id'
# 繪製雷達圖
plot_tw_stock_radar(portfolio=["1580"], feats=None,
                    mode="line_polar", line_polar_fill='toself',
                    cut_bins=4, period=None, title=None, custom_data=custom_data)

In [ ]:
plot_tw_stock_radar(portfolio=["3218"], feats=None,
                    mode="line_polar", line_polar_fill='toself',
                    cut_bins=4, period=None, title=None, custom_data=c)

In [ ]:
plot_tw_stock_radar(portfolio=["1580"], feats=None, mode="line_polar", line_polar_fill='toself', cut_bins=4, period=None, title=None, custom_data=c)

## 產業分群

In [ ]:
return_dataset = {}
ret = []

# 取出產業題材
category = ['光電業', '其他', '其他電子業', '化學工業', '半導體', '塑膠工業', '建材營造', '橡膠工業', '水泥工業', '汽車工業', '油電燃氣業', '生技醫療', '生技醫療業', '紡織纖維', '航運業', '貿易百貨', '資訊服務業', '農業科技', '通信網路業', '造紙工業', '金融', '電器電纜', '電子通路業', '電子零組件', '電機機械', '電腦及週邊', '食品工業']
for c in category:
  with data.universe('TSE_OTC', c):
    print(c)
    rev = data.get('monthly_revenue:當月營收')
    monthly_revenue_grow = data.get("monthly_revenue:去年同月增減(%)")
    operating_profit = data.get("fundamental_features:營業利益率")#.deadline()
    net_income = data.get('fundamental_features:歸屬母公司淨利')#.deadline()
    net_income_r = data.get('fundamental_features:稅後淨利成長率')
    net_income_ratio = data.get('fundamental_features:稅後淨利率')
    stock = data.get('financial_statement:存貨')
    stock_turnover = data.get('fundamental_features:存貨週轉率').replace([np.nan, np.inf], 0)
    season_revenue = data.get('financial_statement:營業收入淨額')

    eps = data.get('financial_statement:每股盈餘')#.deadline()
    df1 = data.get('financial_statement:投資活動之淨現金流入_流出')
    df2 = data.get('financial_statement:營業活動之淨現金流入_流出')
    fcf = (df1 + df2)#.rolling(4).mean()
    pe = data.get('price_earning_ratio:本益比')

    monthly_revenue_grow_aa = ((monthly_revenue_grow > 0).sustain(6)) & (monthly_revenue_grow.average(6) > 25) & (monthly_revenue_grow.rise())
    monthly_revenue_grow_a1 = ((monthly_revenue_grow > 0).sustain(6)) & ((monthly_revenue_grow.average(6) <= 25) & (monthly_revenue_grow.average(6) > 10)) & (monthly_revenue_grow.rise())
    monthly_revenue_grow_a2 = ((monthly_revenue_grow > 0).sustain(6)) & (monthly_revenue_grow.average(6) > 25) & (monthly_revenue_grow < monthly_revenue_grow.shift()) & (monthly_revenue_grow.shift() > monthly_revenue_grow.shift(2)) & (monthly_revenue_grow.diff() / monthly_revenue_grow.shift().abs() * 100 > -50)#(monthly_revenue_grow.pct_change(fill_method=None) > -0.5)
    monthly_revenue_grow_a = monthly_revenue_grow_a1 | monthly_revenue_grow_a2

    monthly_revenue_grow_bb = (monthly_revenue_grow.rolling(6).min() <= 0) & ~(monthly_revenue_grow.fall().sustain(2)) & (monthly_revenue_grow > 0) & (monthly_revenue_grow.average(6) > 0)#~(monthly_revenue_grow_b | monthly_revenue_grow_c)
    monthly_revenue_grow_b = ((monthly_revenue_grow.average(6) > 0)) & (monthly_revenue_grow.fall().sustain(2)) & (monthly_revenue_grow > 0)
    monthly_revenue_grow_c = (monthly_revenue_grow.average(6) < 0) | (monthly_revenue_grow < 0)

    empty_df = dataframe.FinlabDataFrame(pd.DataFrame(1, columns=monthly_revenue_grow.columns, index=monthly_revenue_grow.index))
    empty_df &= ~(monthly_revenue_grow_aa | monthly_revenue_grow_a | monthly_revenue_grow_bb | monthly_revenue_grow_b | monthly_revenue_grow_c)
    monthly_revenue_grow_bb |= empty_df

    operating_profit_stable = (((operating_profit.diff() / operating_profit.shift().abs()).rolling(3).min()) * 100) > -20

    operating_profit_aa_1 = operating_profit_stable & (operating_profit.average(4) > 15)
    operating_profit_aa_2 = operating_profit_stable & ((operating_profit.average(4) <= 15) & ((operating_profit.average(4) > 10)) & operating_profit.rise())
    operating_profit_aa = operating_profit_aa_1 | operating_profit_aa_2

    operating_profit_a_1 = operating_profit_stable & ((operating_profit.average(4) <= 15) & (operating_profit.average(4) > 10)) & operating_profit.fall() # 考慮到 AA_2 條件，A_1 會涵蓋其包含的股票，故排除最近一季呈現上升趨勢
    operating_profit_a_2 = operating_profit_stable & ((operating_profit.average(4) <= 10) & (operating_profit.average(4) > 5)) & (operating_profit.rise())
    operating_profit_a =  operating_profit_a_1 | operating_profit_a_2

    operating_profit_bb = (~operating_profit_stable) & ((operating_profit.diff()/ operating_profit.shift().abs()) * 100 > -20) & (operating_profit.average(4) > 5) & (operating_profit > 0) # 考慮到 B 條件，不論漲跌幅，過去四季平均營益率皆在 5% 以下。BB 條件須排除
                                                                                                            # 考慮到 C 條件，最近一季為負。BB 條件須排除
    operating_profit_b_1 = ((operating_profit.diff()/ operating_profit.shift().abs()) * 100 <= -20)# & (operating_profit.average(4) > 0) # 考慮到 B_2 條件，不論帳跌幅，過去四季平均在 5% 以下。B_1 條件排除過去四季平均在 5% 以下條件。
    operating_profit_b_2 = (operating_profit.average(4) <= 5) #& (operating_profit.average(4) > 0) # 考慮到 C 條件，過去四季平均為負。 避免股票同時屬於 B 條件與 C 條件，新增過去四季平均大於 0 的條件
    operating_profit_b = (operating_profit_b_1 | operating_profit_b_2) & (operating_profit > 0 ) # 考慮到 C 條件，最近一季為負。這裡避免 B 條件被歸納到 C，新增最近一季為正的條件
    operating_profit_b &= (operating_profit.average(4) > 0)

    operating_profit_c = (operating_profit.average(4) <= 0) | (operating_profit <= 0)

    empty_df = dataframe.FinlabDataFrame(pd.DataFrame(1, columns=operating_profit_stable.columns, index=operating_profit_stable.index))
    empty_df &= ~(operating_profit_aa | operating_profit_a | operating_profit_bb | operating_profit_b | operating_profit_c)
    operating_profit_bb |= empty_df

    net_income_grow = net_income_r#(net_income.diff(4) / net_income.shift(4).abs() * 100)
    net_income_grow_stable = net_income_grow.pct_change(fill_method=None) * 100 > -50

    net_income_grow_aa_1 = ((net_income_grow > 0).sustain(3)) & (net_income_grow.rise())
    net_income_grow_aa_2 = (net_income_grow > 50).sustain(3)
    net_income_grow_aa = (net_income_grow_aa_1 | net_income_grow_aa_2)

    net_income_grow_a = (((net_income_grow > 0).sustain(2)) & net_income_grow_stable) & ~(net_income_grow_aa)#~((net_income_grow > 0).sustain(3))
    net_income_grow_a &= ~((net_income_grow < 0).sustain(4, 2)) # 考慮到 B_2 條件，A 須排除最近四季出現兩季負數的情況
    net_income_grow_a &= ~(net_income_grow.fall().sustain(2))

    net_income_grow_bb_1 = ((net_income_grow > 0).sustain(2)) & ~(net_income_grow_stable)
    net_income_grow_bb_1 &= ~((net_income_grow > 50).sustain(3))  # BB_1 的近兩季為正但最近一季出現 50% 跌幅會涵蓋符合 AA_2 的股票，故須排除近三季節大於 50% 的情況
    net_income_grow_bb_1 &= ~(net_income_grow.fall().sustain(2)) # 考慮到 B3 條件，BB_1 須排除連續三季下降的情況
    net_income_grow_bb_2 = (net_income_grow.shift() < 0) & (net_income_grow > 0)
    net_income_grow_bb = (net_income_grow_bb_1 | net_income_grow_bb_2)
    net_income_grow_bb &= (net_income_grow > 0) # 考慮到 B_1 條件，BB 須排除最近一季出現負數的情況
    net_income_grow_bb &= ~((net_income_grow < 0).sustain(4, 2))  # 考慮到 B_2 條件，BB 須排除最近四季出現兩季負數的情況

    net_income_grow_b_1 = net_income_grow < 0
    net_income_grow_b_2 = (net_income_grow < 0).sustain(4, 2)
    net_income_grow_b_3 = (net_income_grow.fall().sustain(2)) & (net_income_grow < 50)
    net_income_grow_b = (net_income_grow_b_1 | net_income_grow_b_2 | net_income_grow_b_3) & ~((net_income_grow < 0).sustain(2)) # 靠慮到 C 條件， B 條件須排除近兩季為負數的情況

    net_income_grow_c = (net_income_grow < 0).sustain(2)

    eps_aa = (eps.rolling(4).sum() > 5) & (eps > 0)
    eps_a = (eps.rolling(4).sum() <= 5) & (eps.rolling(4).sum() > 3) & (eps > 0)
    eps_bb = (eps.rolling(4).sum() <= 3) & (eps.rolling(4).sum() > 1) & (eps > 0)
    eps_b_1 = (eps.rolling(4).sum() <= 1)
    eps_b_2 = (eps < 0)
    eps_b = (eps_b_1 | eps_b_2) & (eps.rolling(4).sum() > 0)
    eps_c = eps.rolling(4).sum() <= 0

    stock_low = (stock_turnover <= 0) | (stock_turnover > 10) | ((stock / season_revenue) <= 0.04)
    stock_turnover_stable = (stock_turnover.diff() / stock_turnover.shift().abs()).rolling(3).min() * 100 > -20
    stock_turnover_cumulate_loss_gt_m20 = (stock_turnover.fall().sustain(3, 2)) & (stock_turnover.pct_change()[stock_turnover.pct_change() < 0].rolling(2).sum() * 100 < -20)

    stock_turnover_aa = (~stock_low) & stock_turnover_stable & (stock_turnover.average(4) > 1.5) & ~(stock_turnover_cumulate_loss_gt_m20)
    stock_turnover_a = (~stock_low) & stock_turnover_stable & (stock_turnover.average(4) <= 1.5) & ~(stock_turnover_cumulate_loss_gt_m20)
    stock_turnover_bb = (~stock_low) & stock_turnover_cumulate_loss_gt_m20 & (stock_turnover_stable)
    stock_turnover_b = (~stock_low) & ~(stock_turnover_stable) & (stock_turnover.pct_change() * 100 > -20)
    stock_turnover_c = (~stock_low) & (stock_turnover.pct_change() * 100 <= -20)

    fcf_aa = fcf.rolling(6).min() > 0
    fcf_a = (fcf.rolling(6).sum() > 0) & (fcf.rolling(4).sum() > 0) & ~(fcf_aa)
    fcf_bb = (fcf.rolling(6).sum() <= 0) & (fcf.rolling(4).sum() > 0)# & ~(fcf.rolling(6).min() > 0)
    fcf_b = (fcf.rolling(6).sum() > 0) & (fcf.rolling(4).sum() <= 0)# & ~(fcf.rolling(6).min() > 0)
    fcf_c = (fcf.rolling(6).sum() <= 0) & (fcf.rolling(4).sum() <= 0)# & ~(fcf.rolling(6).min() > 0)

    substract_level_c = monthly_revenue_grow_c | operating_profit_c | net_income_grow_c | eps_c | fcf_c | stock_turnover_c
    select_low_stock = ~substract_level_c * stock_low * ((4 * (monthly_revenue_grow_aa + operating_profit_aa + net_income_grow_aa + eps_aa + fcf_aa) +
                3 * (monthly_revenue_grow_a + operating_profit_a + net_income_grow_a + eps_a + fcf_a) +
                2 * (monthly_revenue_grow_bb + operating_profit_bb + net_income_grow_bb + eps_bb + fcf_bb) +
                1 * (monthly_revenue_grow_b + operating_profit_b + net_income_grow_b + eps_b + fcf_b)) / 5)
    select_normal = ~substract_level_c * ~(stock_low) * ((4 * (monthly_revenue_grow_aa + operating_profit_aa + net_income_grow_aa + eps_aa + stock_turnover_aa + fcf_aa) +
              3 * (monthly_revenue_grow_a + operating_profit_a + net_income_grow_a + eps_a + stock_turnover_a + fcf_a) +
              2 * (monthly_revenue_grow_bb + operating_profit_bb + net_income_grow_bb + eps_bb + stock_turnover_bb + fcf_bb) +
              1 * (monthly_revenue_grow_b + operating_profit_b + net_income_grow_b + eps_b + stock_turnover_b + fcf_b)) / 6)

    s = (select_normal + select_low_stock).fillna(0)#.reindex(rev.index_str_to_date().index, method='ffill')
    p = ((s > 3) & (s - s.shift() >= -0.3))
    r = backtest.sim(position=p, upload=False)
    return_series = r.creturn
    if r.creturn[-1] >= 4:
      return_dataset[f"{c}"] = return_series
      ret.append({'category': c, 'return':return_series[-1]})

光電業


其他
其他電子業
化學工業
半導體
塑膠工業
建材營造
橡膠工業
水泥工業
汽車工業
油電燃氣業
生技醫療
生技醫療業
紡織纖維
航運業
貿易百貨
資訊服務業
農業科技
通信網路業
造紙工業
金融
電器電纜
電子通路業
電子零組件
電機機械
電腦及週邊
食品工業


In [ ]:
#ret.append({'return':return_series[-1]})

df = pd.DataFrame(ret)

fig = px.bar(df, height=600, x='category', y='return', color='return', title='Backtest by categroy')

fig.show()